# 🚀 Miniprojeto - Consultas BigQuery

Este notebook contém todos os scripts SQL utilizados no projeto:
- Criação de tabelas
- Inserção de dados
- Criação de views
- Consultas de análise


## 📊 Criação de Tabelas





In [ ]:
-- 01_create_tables.sql
CREATE TABLE Livraria_DevSaber_1_10.clientes (
  cliente_id STRING,
  nome STRING,
  estado STRING
);

CREATE TABLE Livraria_DevSaber_1_10.produtos (
  produto_id STRING,
  nome_produto STRING,
  categoria STRING,
  preco NUMERIC
);

CREATE TABLE Livraria_DevSaber_1_10.vendas (
  venda_id STRING,
  cliente_id STRING,
  produto_id STRING,
  quantidade INT64,
  data_venda DATE
);


## 📊 Inserção de Dados

In [ ]:
-- 02_insert_data.sql
INSERT INTO Livraria_DevSaber_1_10.clientes
(cliente_id, nome, estado) VALUES
('C001', 'Ana Silva', 'SP'),
('C002', 'Bruno Costa', 'RJ'),
('C003', 'Carla Dias', 'MG'),
('C004', 'Daniel Souza', 'RS');

INSERT INTO Livraria_DevSaber_1_10.produtos
(produto_id, nome_produto, categoria, preco) VALUES
('P001', 'Livro A', 'Ficção', 50),
('P002', 'Livro B', 'Ficção', 70),
('P003', 'Livro C', 'Negócios', 100),
('P004', 'Livro D', 'Educação', 80);

INSERT INTO Livraria_DevSaber_1_10.vendas
(venda_id, cliente_id, produto_id, quantidade, data_venda) VALUES
('V001', 'C001', 'P001', 1, '2023-06-01'),
('V002', 'C001', 'P002', 1, '2023-07-15'),
('V003', 'C002', 'P003', 1, '2023-08-20'),
('V004', 'C003', 'P004', 1, '2023-08-22'),
('V005', 'C004', 'P001', 1, '2023-09-05');
-- 02_insert_data.sql
INSERT INTO Livraria_DevSaber_1_10.clientes
(cliente_id, nome, estado) VALUES
('C001', 'Ana Silva', 'SP'),
('C002', 'Bruno Costa', 'RJ'),
('C003', 'Carla Dias', 'MG'),
('C004', 'Daniel Souza', 'RS');

INSERT INTO Livraria_DevSaber_1_10.produtos
(produto_id, nome_produto, categoria, preco) VALUES
('P001', 'Livro A', 'Ficção', 50),
('P002', 'Livro B', 'Ficção', 70),
('P003', 'Livro C', 'Negócios', 100),
('P004', 'Livro D', 'Educação', 80);

INSERT INTO Livraria_DevSaber_1_10.vendas
(venda_id, cliente_id, produto_id, quantidade, data_venda) VALUES
('V001', 'C001', 'P001', 1, '2023-06-01'),
('V002', 'C001', 'P002', 1, '2023-07-15'),
('V003', 'C002', 'P003', 1, '2023-08-20'),
('V004', 'C003', 'P004', 1, '2023-08-22'),
('V005', 'C004', 'P001', 1, '2023-09-05');


##  📊 Criação das Views

In [ ]:
-- 03_views.sql
CREATE OR REPLACE VIEW Livraria_DevSaber_1_10.vw_vendas_detalhes AS
SELECT
  v.venda_id,
  v.data_venda,
  c.cliente_id,
  c.nome AS nome_cliente,
  c.estado AS estado_cliente,
  p.produto_id,
  p.nome_produto,
  p.categoria AS categoria_produto,
  v.quantidade,
  p.preco,
  v.quantidade * p.preco AS total_venda
FROM Livraria_DevSaber_1_10.vendas v
JOIN Livraria_DevSaber_1_10.clientes c ON v.cliente_id = c.cliente_id
JOIN Livraria_DevSaber_1_10.produtos p ON v.produto_id = p.produto_id;


## 📊 Consultas de Análise

In [ ]:
-- 04_analysis_queries.sql

-- Total de vendas por cliente
SELECT nome_cliente, SUM(total_venda) AS total_gasto
FROM Livraria_DevSaber_1_10.vw_vendas_detalhes
GROUP BY nome_cliente
ORDER BY total_gasto DESC;

-- Produtos mais vendidos
SELECT nome_produto, SUM(quantidade) AS total_vendido
FROM Livraria_DevSaber_1_10.vw_vendas_detalhes
GROUP BY nome_produto
ORDER BY total_vendido DESC;

-- Receita por mês
SELECT FORMAT_DATE("%Y-%m", data_venda) AS ano_mes,
       SUM(total_venda) AS receita_total
FROM Livraria_DevSaber_1_10.vw_vendas_detalhes
GROUP BY ano_mes
ORDER BY ano_mes;

-- Pareto
WITH receita_total AS (
  SELECT SUM(total_venda) AS receita_geral
  FROM Livraria_DevSaber_1_10.vw_vendas_detalhes
),
clientes_receita AS (
  SELECT cliente_id, nome_cliente, SUM(total_venda) AS receita_cliente
  FROM Livraria_DevSaber_1_10.vw_vendas_detalhes
  GROUP BY cliente_id, nome_cliente
  ORDER BY receita_cliente DESC
)
SELECT
  c.*,
  r.receita_geral,
  SUM(c.receita_cliente) OVER (ORDER BY receita_cliente DESC) AS receita_acumulada,
  (SUM(c.receita_cliente) OVER (ORDER BY receita_cliente DESC)/r.receita_geral) AS percentual_acumulado
FROM clientes_receita c
CROSS JOIN receita_total r;
